# Stream tweets

In [ ]:
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [2]:
access_token = "634676997-Kz23NPQNf4YCTlDrrrTAQrDrnO0pr00qq8hPAKy0"
access_token_secret = "thRSBkUJnzg5jUPhkOnT9rRproZCfvpntdU49QLhtNtlE"
consumer_key = "eCjoqU4GxbipyQ3AcPbbWW1LE"
consumer_secret = "WHGqS4EaCqXeHftgMvrqMaeYVtcaxpE8NWAgcIRxevtPag622O"

In [3]:
## test code to check authentication
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
api = tweepy.API(auth)

for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    print(status.text)

.@amyschumer made her return to #SNL this week for her second tenure as host. What did you think of the comedian's… https://t.co/P2K9RCZzRf
Wow. I love this! https://t.co/lcj5lxMj5l
Happy Mother's Day to the extraordinary mothers of our community (especially you, Mom)! https://t.co/Z84IPN7V7m
In case anyone needed a reminder, don't become complacent by what the polling says 

https://t.co/PHQwxKvM4Q https://t.co/wAYmosWHvI
RT @jackiedobak: Causally just joined over 500,000 alumni #RU2018 https://t.co/OoT1t6LMtJ
RT @FirstYearRUSOE: Congratulations to Nelson Leal, @UCCweb grad, graduation with a degree in Bioenvironmental engineering, Magna Cum Laude…
RT @FirstYearRUSOE: Graduation Doggy with his own cap at @RutgersSOE graduation. #RU2018 #dogsoftwitter #GraduationDay https://t.co/JmKHJns…
RT @namritas14: I finally have a Bachelor’s degree to my name and I can’t wait to see where my journey takes me next. Rutgers, I am forever…
RT @chrissgardner: Made it to HFPA dinner in time to see #Ca

In [ ]:
class StdOutListener(StreamListener):

   def on_data(self, data):
    try:
        with open('healthcare_tweets.txt', 'a') as my_file:
            my_file.write(data)
    except BaseException:
        print('Error')
        pass

   def on_error(self, status):
       print (status.text)


if __name__ == '__main__':

   #This handles Twitter authetification and the connection to Twitter Streaming API
   l = StdOutListener()
   auth = OAuthHandler(consumer_key, consumer_secret)
   auth.set_access_token(access_token, access_token_secret)
   stream = Stream(auth, l)

   #This line filter Twitter Streams to capture data by the keywords: 'python', 'javascript', 'ruby'
   stream.filter(track=['healthcare','obamacare','trumpcare','AHCA','ACA'])

# Read collected tweets for sentiment analysis

### data cleaning

In [1]:
#read the tweets into a list
import json
import pandas as pd
import matplotlib.pyplot as plt

tweets_data = []
tweets_file = open('healthcare_tweets_1.txt', "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue
        
print('collected %f tweets'%len(tweets_data))

collected 240034.000000 tweets


In [2]:
#remove empty tweets
tweets = pd.DataFrame(tweets_data)
tweets=tweets[['id_str','user','lang','favorited','retweeted','text']].dropna(axis=0,how='all').reset_index()
print('removed all the empty tweets, now we have %f tweets'%len(tweets))

removed all the empty tweets, now we have 240000.000000 tweets


In [3]:
#choose only users from US
user_list=pd.DataFrame(tweets['user'].values.tolist())
user_list=user_list[['location','id_str','followers_count','name','screen_name']]
tweets_all = pd.concat([tweets, user_list], axis=1).drop(['user'],axis=1)
tweets_all=tweets_all[tweets_all['lang']=='en']
tweets_all=tweets_all[pd.notna(tweets_all['location'])]

states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

state_abbr=list(states.keys())
state_full=list(states.values())

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def get_state (row): #parse the location data to obtain only the US ones
    abbr=intersection(row['location'].split(', '),state_abbr)
    full=intersection(row['location'].split(', '),state_full)
    abbr_1=intersection(row['location'].split(),state_abbr)
    full_1=intersection(row['location'].split(),state_full)
    
    if len(abbr)!=0:
        return abbr[0]
    elif len(abbr_1)!=0:
        return abbr_1[0]
    elif len(full)!=0:
        return state_abbr[state_full.index(full[0])]
    elif len(full_1)!=0:
        return state_abbr[state_full.index(full_1[0])]
    return None
            
tweets_all['state']=tweets_all.apply(lambda row: get_state(row),axis=1)

tweets_all=tweets_all[pd.notna(tweets_all['state'])]


### plot the number of tweets by state

In [4]:
# plot the number of tweets sent out by state
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

init_notebook_mode(connected=True)

df= tweets_all.groupby('state').agg(['count']).reset_index()

scl = [[0.0, '#67a9cf'],[0.1, '#d1e5f0'],[0.3, '#fddbc7'], 
       [0.5, '#fddbc7'],[1, '#b2182b']]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df.state,
        z = df['location']['count'],
        locationmode = 'USA-states',
        text = df.state,
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Tweet Count")
        )
       ]

layout = dict(
        title = 'Number of tweets in 24 hours',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

### tweet sentiment analysis

In [5]:
#tweet sentiment analysis and visualize the results
import re
from textblob import TextBlob

def clean_tweet(tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
    
def get_tweet_sentiment(row):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(clean_tweet(row['text']))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
        
tweets_all['sentiment']=tweets_all.apply(lambda row: get_tweet_sentiment (row),axis=1)

df=tweets_all.sort_values(by=['followers_count'],ascending=False)
positive=df[df['sentiment']=='positive'].head()['text'].values
neutral=df[df['sentiment']=='neutral'].head()['text'].values
negative=df[df['sentiment']=='negative'].head()['text'].values

print ('Positive tweeet examples:')
for i in range(len(positive)):
    print(positive[i])
    
print ()
print ('Neutral tweeet examples:')
for i in range(len(neutral)):
    print(positive[i])  
    
print ()
print ('Negative tweeet examples:')
for i in range(len(negative)):
    print(positive[i])  


Positive tweeet examples:
Healthcare accounts for nearly 18% of our economy – explore how this critical sector is reshaping our world… https://t.co/hLeIBaY2LW
Democrats are confidently running on Obamacare for the first time in a decade https://t.co/vyaChmzyfu https://t.co/jOETPZ6Nq8
RT @legalaidtech: "In the policy world, wonks spend decades debating the finer points of healthcare and social spending, only to be wholly…
RT @eugenegu: Not even America’s best hospital can handle 2000 patients critically injured by live ammo. A triage situation would be in pla…
Amelia Ballard spent many years of her childhood battling leukemia at Children's Healthcare of Atlanta. Now she's c… https://t.co/Pfa7J0FsZT

Neutral tweeet examples:
Healthcare accounts for nearly 18% of our economy – explore how this critical sector is reshaping our world… https://t.co/hLeIBaY2LW
Democrats are confidently running on Obamacare for the first time in a decade https://t.co/vyaChmzyfu https://t.co/jOETPZ6Nq8
RT @lega

In [6]:
# plot the result of sentiment analysis
import numpy as np
from operator import add

df= tweets_all[['state','sentiment','name']].groupby(['state','sentiment']).agg('count').reset_index()

#find the sum of tweets
def sum(row):
    return row['negative_ct']+row['neutral_ct']+row['positive_ct']

df= tweets_all[['state','sentiment','name']].groupby(['state','sentiment']).agg('count').reset_index()
df_neg=df[df['sentiment']=='negative'].drop(['sentiment'],axis=1).rename(columns={"name": "negative_ct"}).set_index('state')
df_neut=df[df['sentiment']=='neutral'].drop(['sentiment'],axis=1).rename(columns={"name": "neutral_ct"}).set_index('state')
df_pos=df[df['sentiment']=='positive'].drop(['sentiment'],axis=1).rename(columns={"name": "positive_ct"}).set_index('state')
df=pd.merge(pd.merge(df_neg.reset_index(), df_neut.reset_index(), on='state'),df_pos.reset_index(), on='state')
df['sum_tweets']=df.apply(lambda row:sum(row),axis=1)
df=df.sort_values(by=['sum_tweets'],ascending=False)

states=df['state'].values
num_of_states=len(states)

ind=np.arange(num_of_states)
negative_ct=df['negative_ct'].values
neutral_ct=df['neutral_ct'].values
positive_ct=df['positive_ct'].values

trace1 = go.Bar(
    x=states,
    y=negative_ct,
    name='Negative'
)
trace2 = go.Bar(
    x=states,
    y=neutral_ct,
    name='Neutral'
)
trace3 = go.Bar(
    x=states,
    y=positive_ct,
    name='Positive'
)

data = [trace1,trace2,trace3]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

### latent sentiment analysis

In [7]:
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.decomposition import TruncatedSVD

#use the same program to clean tweets
def clean_tweet(row):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", row['text']).split())
    

tweets_all['cleaned_tweet']=tweets_all.apply(lambda row: clean_tweet(row),axis=1)
clean_tweets=tweets_all['cleaned_tweet'].values
stopwords = set(w.rstrip() for w in open('stopwords.txt'))

stopwords = stopwords.union({'america', 'healthcare','this','care','however','john','trump','ha',
                             'mccain','you','american','people','retweet'})

wordnet_lemmatizer = WordNetLemmatizer()

def my_tokenizer(s):
    s = s.lower() # downcase
    tokens = nltk.tokenize.word_tokenize(s) # split string into words (tokens)
    tokens = [t for t in tokens if len(t) > 2] # remove short words, they're probably not useful
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # put words into base form
    tokens = [t for t in tokens if t not in stopwords] # remove stopwords
    tokens = [t for t in tokens if not any(c.isdigit() for c in t)] # remove any digits, i.e. "3rd edition"
    return tokens

word_index_map = {}
current_index = 0
all_tokens = []
all_tweets = []
index_word_map = []
error_count = 0

for tweet in clean_tweets:
    try:
        tweet = tweet.encode('ascii', 'ignore').decode('utf-8') # this will throw exception if bad characters
        all_tweets.append(tweet)
        tokens = my_tokenizer(tweet)
        all_tokens.append(tokens)
        for token in tokens:
            if token not in word_index_map:
                word_index_map[token] = current_index
                current_index += 1
                index_word_map.append(token)
    except Exception as e:
        print(e)
        print(tweet)
        error_count += 1


print("Number of errors parsing file:", error_count, "number of lines in file:", len(clean_tweets))
if error_count == len(clean_tweets):
    print("There is no data to do anything with! Quitting...")
    exit()
    
def tokens_to_vector(tokens):
    x = np.zeros(len(word_index_map))
    for t in tokens:
        i = word_index_map[t]
        x[i] = 1
    return x

N = len(all_tokens)
D = len(word_index_map)
X = np.zeros((D, N)) # terms will go along rows, documents along columns
i = 0
for tokens in all_tokens:
    X[:,i] = tokens_to_vector(tokens)
    i += 1

def main():
    svd = TruncatedSVD()
    Z = svd.fit_transform(X)
    # Create a trace
    trace = go.Scatter(
        x = Z[:,0],
        y = Z[:,1],
        mode = 'markers+text',
        text=index_word_map,
        textposition='bottom'
    )

    data = [trace]

    # Plot and embed in ipython notebook!
    iplot(data, filename='basic-scatter')

if __name__ == '__main__':
    main()

Number of errors parsing file: 0 number of lines in file: 20114
